In [1]:
import numpy as np
import pandas as pd
import string
import re
import codecs
from gensim.models import word2vec
import tqdm

In [2]:
with codecs.open('./hin-eng/hin.txt', encoding='utf-8') as f:
    data1 = f.read()

In [3]:
with codecs.open('./hin-eng/c.clean.hi', encoding='utf-8') as f:
    hi = f.read()
hi = hi.split('\n') 

with codecs.open('./hin-eng/c.clean.en', encoding='utf-8') as f:
    en = f.read()
en = en.split('\n') 

In [4]:
print len(hi)
print len(en)

92978
92978


In [5]:
def cleaner(line):
    line = line.replace("\t"," ")
    line = re.sub(r"\'ve", " have", line)
    line= re.sub(r"\'s", " us", line)
    line= re.sub(r"\'ll", " will", line)
    line= re.sub(r"\'re", " are", line)
    line = re.sub(r"can't", "cannot ", line)
    line = re.sub(r"n't", " not", line)
    line = re.sub(r"\'d", " would ", line)
    line = re.sub(r"\'ve", " have ", line)
    line = re.sub(r"\'m ", " am", line)
    for punct in string.punctuation:
        line = line.replace(punct," {} ".format(punct))
    line = " ".join(line.split())
    return line

In [6]:
pairs = data1.split('\n')
english_sentences = []
hindi_sentences = []
for i, pair in enumerate(pairs):
    pair_split = pair.split('\t')
    if len(pair_split)!= 2:
        continue
    english = pair_split[0].lower()
    hindi = pair_split[1]
    #punctuation_table = english.maketrans({i:None for i in string.punctuation})
    english = cleaner(english)
    hindi = cleaner(hindi)
    english= " ".join([x.strip(string.punctuation+u"।") for x in english.split()])
    hindi= " ".join([x.strip(string.punctuation+u"।") for x in hindi.split()])
    english_sentences.append(english.strip())
    hindi_sentences.append(hindi.strip())

In [7]:
for i in zip(hi,en):
    english = i[1].lower()
    hindi = i[0]
    english = cleaner(english)
    hindi = cleaner(hindi)
    english= " ".join([x.strip(string.punctuation+u"।") for x in english.split()])
    hindi= " ".join([x.strip(string.punctuation+u"।") for x in hindi.split()])
    english_sentences.append(english.strip())
    hindi_sentences.append(hindi.strip())

In [8]:
print len(english_sentences)
print len(hindi_sentences)

95844
95844


In [9]:
#with codecs.open('./hin-eng/englsih2.txt', 'w',encoding='utf-8') as filehandle:  
#    for listitem in english_sentences:
#        filehandle.write('%s\n' % listitem)
        
#with codecs.open('./hin-eng/hindi2.txt', 'w',encoding='utf-8') as filehandle:  
#    for listitem in hindi_sentences:
#        filehandle.write('%s\n' % listitem)

In [10]:
test_english = english_sentences[-5000:]
test_hindi = hindi_sentences[-5000:]

In [11]:
print len(test_english)
print len(test_hindi)

5000
5000


In [12]:
english_sentences = english_sentences[:-500]
hindi_sentences = hindi_sentences[:-500]

In [13]:
print len(english_sentences)
print len(hindi_sentences)

95344
95344


### Word_embedding

In [14]:
en_w2v = word2vec.Word2Vec.load('./hin-eng/english_model')
hi_w2v = word2vec.Word2Vec.load('./hin-eng/hindi_model')

In [15]:
en_id = {}
hi_id = {}
en_id['<pad>'] = 0
en_id['<start>'] = 1
en_id['<unk>'] = 2
en_id['<end>'] = 3

hi_id['<pad>'] = 0
hi_id['<start>'] = 1
hi_id['<unk>'] = 2
hi_id['<end>'] = 3

for k, v in en_w2v.wv.vocab.items():
    en_id[k] = v.index+4
    
#en_id = dict([(k, v.index+len(en_id)) for k, v in en_w2v.wv.vocab.items()])
id_en = dict([(v, k) for k, v in en_id.items()])
for k, v in hi_w2v.wv.vocab.items():
    hi_id[k] = v.index+4
#hi_id = dict([(k, v.index+len(hi_id)) for k, v in hi_w2v.wv.vocab.items()])
id_hi = dict([(v, k) for k, v in hi_id.items()])

In [16]:
en_emb = en_w2v.wv.syn0
hi_emb = hi_w2v.wv.syn0

/home/shaurya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.
/home/shaurya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [17]:
print en_emb.shape
print hi_emb.shape

(42526, 300)
(29138, 300)


In [18]:
en_vectors = list(np.random.uniform(-0.1, 0.1, (4, 300)))
en_vectors[0] *= 0 # make the padding vector zeros
hi_vectors = list(np.random.uniform(-0.1, 0.1, (4, 300)))
hi_vectors[0] *= 0 # make the padding vector zeros

In [19]:
en_emb = np.insert(en_emb,0,en_vectors[0],axis=0)
en_emb = np.insert(en_emb,1,en_vectors[1],axis=0)
en_emb = np.insert(en_emb,2,en_vectors[2],axis=0)
en_emb = np.insert(en_emb,3,en_vectors[3],axis=0)

hi_emb = np.insert(hi_emb,0,hi_vectors[0],axis=0)
hi_emb = np.insert(hi_emb,1,hi_vectors[1],axis=0)
hi_emb = np.insert(hi_emb,2,hi_vectors[2],axis=0)
hi_emb = np.insert(hi_emb,3,hi_vectors[3],axis=0)

--------------

In [20]:
print len(english_sentences), len(hindi_sentences)

95344 95344


In [21]:
max_en_length = 0
for sentence in english_sentences:
    length = len(sentence.split())
    max_en_length = max(max_en_length, length)
print("The longest english sentence in our dataset is:", max_en_length) 

('The longest english sentence in our dataset is:', 57)


In [22]:
max_hi_length = 0
for sentence in hindi_sentences:
    length = len(sentence.split())
    max_hi_length = max(max_hi_length, length)
print("The longest english sentence in our dataset is:", max_hi_length) 

('The longest english sentence in our dataset is:', 56)


In [23]:
max_seq_length = max(max_hi_length, max_en_length) + 2
seq_length = max_seq_length

In [24]:
en_word_count = {}
hi_word_count = {}

for sentence in english_sentences:
    for word in sentence.split():
        if word in en_word_count:
            en_word_count[word] +=1
        else:
            en_word_count[word] = 1
            
for sentence in hindi_sentences:
    for word in sentence.split():
        if word in hi_word_count:
            hi_word_count[word] +=1
        else:
            hi_word_count[word] = 1

In [25]:
print('Number of unique English words:', len(en_word_count))
print('Number of unique Hindi words:', len(hi_word_count))

('Number of unique English words:', 20007)
('Number of unique Hindi words:', 23478)


In [26]:
hi_word2idx = {k:v+4 for v, k in enumerate(hi_word_count.keys())}
en_word2idx = {k:v+4 for v, k in enumerate(en_word_count.keys())}

In [27]:
hi_word_count['<start>'] = len(hindi_sentences)
hi_word_count['<end>'] = len(hindi_sentences)
en_word_count['<start>'] = len(english_sentences)
en_word_count['<end>'] = len(english_sentences)

In [28]:
hi_word2idx['<pad>'] = 0
hi_word2idx['<start>'] = 1
hi_word2idx['<end>'] = 2
hi_word2idx['<unk>'] = 3

en_word2idx['<pad>'] = 0
en_word2idx['<start>'] = 1
en_word2idx['<end>'] = 2
en_word2idx['<unk>'] = 3

In [29]:
word2idx_en = dict([(v, k) for k, v in en_word2idx.items()])
word2idx_hi = dict([(v, k) for k, v in hi_word2idx.items()])

In [30]:
#en_vectors = list(np.random.uniform(-0.1, 0.1, (4, 300)))
#en_vectors[0] *= 0 # make the padding vector zeros
#hi_vectors = list(np.random.uniform(-0.1, 0.1, (4, 300)))
#hi_vectors[0] *= 0 # make the padding vector zeros

In [31]:
en_vectors= []
hi_vectors = []
for v,k in word2idx_en.items():
    try:
        en_vectors.append(en_w2v.wv.word_vec(k).reshape(-1,300))
    except KeyError:
        en_vectors.append(np.random.uniform(-0.1, 0.1, (1, 300)))
        print k
    
for v,k in word2idx_hi.items():
    try:
        hi_vectors.append(hi_w2v.wv.word_vec(k).reshape(-1,300))
    except KeyError:
        hi_vectors.append(np.random.uniform(-0.1, 0.1, (1, 300)))
        print k

<pad>
<start>
<end>
<unk>
bunnies
worshiped
clearinghouses
reichsmarks
centimeter
yusan
zacjelit
jairam
scold
outwit
diver
spitelout
rossetti
revelers
voyeur
iit
circuitry
crotch
oooh
najeeb
georgetown
morisan
968
scrapes
scraper
pouty
amhanging
washerman
linge
groupie
amdabbling
peppercorns
herb
swag
punks
sattlu
chino
conversational
appendence
jocks
amriid
climber

preheat
santona
maisie
gianna
schwing
catchy
andrex
cannibal
dispatcher
amjealous
sideface
exorcism
hols
roosters
takeout
goddammit
zhoan
mustakeen
lfwe
ryutaro
leaped
overstress
vraèao
matilde
amway
centennial
britania
pints
fonder
swivel
salamanders
maegor
brandon
playback
barbra
bhairavakona
possum
dough
rodrigue
turd
ammo
viserys
hegemon
pallid
profaned
malfunctioned
iletyou
bling
arrestable
closers
munny
thrones
rescuee
vanja
hahahahahaaa
usman
towered
satchel
turnip
whir
rr
dss
villette
plushy
scrubbers
ri
am31
amfeelin
sellout
amabsolutely
tinkles
tarzan
amproud
sys
birdie
robitussin
bookmark
èednost
butjust
reitsc

xy
xt
jugs
wehrmacht
malteser
valka
fl
incinerates
schukov
tusks
naughtier
heartburn
larks
floories
x2
amtouched
hammocks
loverboy
24x7
spareribs
karate
youngstown
unharnassed
crumpets
decompose
browser
skateboarding
grotty
jove
stoners
engin
koldukardarskinoquue
métro
righthemisphere
rso
shallows
amover
bitching
reaving
hots
newsreaders
shithouse
clusterfuck
udaje
slaved
gitte
boggly
posse
portese
hairpin
fletcher
vandervoort
amcool
amrance
thompson
misidentified
amtasking
welders
pastrami
brienne
wabash
yamarashi
synthesized
synapse
amandy
lovebirds
storeys
girdled
keyed
amblind
pvcu
girdles
steakhouses
imposter
icbm
maalox
magliani
hoes
incisor
bender
rakshasa
rollicking
japanesey
rajni
amhealed
prettiest
handpicked
harrass
moans
cakewalk
lasorda
voicemail
cub
spirera
babbling
belinda
amapproaching
synced
bedazzle
hvt
brats
lemaitre
congenita
graff
ambience
amsteve
amant
burgled
romley
ternoon
norinei
suzdr
mareecha
coolest
yi
28pm
stewardess
awhile
medic
arashikage
mahlsdorf
lite
a

dobhyanskaya
snotty
thinnest
macy
sputtering
birthing
homeboy
wally
telemetry
funneled
laforge
bravest
withers
flutters
humps
scribbles
progressiνe
snafu
lymphoblastic
kratom
amcursed
behooves
chitchat
1133
mamihlapinatapai
yanks
feisty
nonoperative
birdcalls
qutub
baratheons
kicker
stares
crossbow
amworthless
bernacles
bloomed
rogaine
armored
changer
okayasu
encirclement
offa
ammija
solenopsis
alchemists
assed
m4
racketeer
opsnog
alwsys
ravishing
pixelate
befriended
myrrh
md
hideout
dodi
perdi
tc
lunge
entertainer
downfield
amfearless
sentries
mermaids
predomi
riina
mornin
loaning
plaits
mohan
calve
ti
santana
bunk
amigo
astronomers
buns
bunt
moisten
eyrie
horoscope
unguarded
backslash
38th
amtrev
hypnotic
smrti
kalimova
clapper
pampered
firecrackers
fealty
howdy
jokul
aishitemasu
vandalize
psspsst
trippin
slutty
manicure
testiest
skirling
victori
tweedledee
testicles
flatteries
amscott
sacré
nik
magneto
gymnasium
blares
sears
gearshift
storch
pounce
stuey
motel
artifacts
cauchoix
und

jump
बेंटले
ऐंठना
Locked
सँभाले
सन्निपात
Palace
RETCHES
कठौता
टीयरगार्टन
Doctor
अल्ज़ाइमर
उद्धारक
मेग्मा
हैक्स
रेगिनाल्ड
फ़्रांसीसी
matchsticks
फ़ायर
जीज़स
फेरारी
विस्परिंग
links
ओकिनावा
कॉन्यैक
फ़ालतू
farm
twoPilot
alibis
FLETC
अफ़्रीकियों
ईव्निंग
कमीज़
आँकड़े
यक्ति
Hexlite
वोह
costume
dancin
सुधारिए
demolitions
गड़ाय
ब्यूटीशियन
Two
कोमपियांग
मिशनो
स्कीइंग
डोमिनोज
बग़ीचे
अर्जंट
रसोईए
Loras
स्केटबोर्डिंग
पेनी
डेला
ज़िपलॉक
शिकंजा
आगंतुक
bitch
शिप्र
बुलंदी
दिखें
craziest
Gears
मुख़्तार
Harpies
Neighs
फुर्तीला
तलना
बख्तरबंद
trialled
एतद्द्वारा
Scythia
टी20
बेवकूफ़ी
संभांलो
Hallyne
3711
इतराने
जबड़ा
प्रलोभित
कैनटीन
spotièem
सिकुड़
निंजा
फोटोग्राफ़ी
शुद्धतम
डोनट्स
फुसफुसाता
तडफ़ता
जबड़े
डिज़ाइनर
Lehnsherr
खर्राटों
केनवरल
Withlock
unbreached
pleeeaaase
shitstorm
पोसना
sheepherders
पुनर्निर्धारित
gibitaka
भेजेगी
erased
बकसुआ
होमो
यूपीवीसी
आकाशगंगा
खुशकिसमत
एमआर
मैयत
आंकड़ों
वेर्नर
बैच
evacuees
पटाएँ
कांची
लुईस
कंसोल
सर्जनों
party
थेरेपी
दौड़ते
दशहरा
Gromyko
आपाराधिक
अस्पष्टीकृत
राफ़ेलसन
Toenail

मीलो
nuns
मिलवा
शिक़ायत
enslaves
1792
हमतक
काजुन
सभाकक्ष
Spa
मीला
दुःस्वप्न
manliness
लूइस
अफ़गानी
अबेनादेर
अस्तित्वहीन
बेंडर
shmuck
खिलना
Farris
coddling
सीबीडी
आज़मा
डिस्प्ले
पूरव
दफनाना
बेजोड़
बोलोक्स
किंवदंती
IAB
किलिक
गाऊँगा
ओबर्हाउस
एमआईटी
परोसा
त्रिक
Brother
fundraisers
टेकऑफ़
टूट्सी
Beginning
तोड़फोड़
alchemists
zaštititmo
टोटम
फ्रैनसीन
keep
क्लिपबोर्ड
ख़ुफ़िया
Reve
फ़ैक्टरी
ड्रीम
attack
मचाना
लेज़्ली
छुपना
clanging
formalities
dyno
चैफ
bed
रेट्रोग्रेड
buggers
गुंडारोधी
Guderian
ढूंढ़ो
आवर्धन
पादन
फटके
पुलीस
ऐमब्यूलेंस
bloodthirsty
ख़रीद
कं
डिस्कोथेक
युद्धाभ्यास
भूमिमार्ग
रेक्स
120
123
124
125
भिनभिनाहट
लेस्ली
कोजक
बिछुड़
she
डिक्की
Neèeš
चिड़ियों
लौटाए
written
प्रग्णालय
Gordian
प्रोग्रामर
कॉर्पोरेट
एथलीटों
निगलो
लिफ़ाफ़े
osjaèam
Vermeer
पँहुचे
mothafucka
username
kicks
Bama
परिंदे
हॉक्स
शर्िमंदा
झल्लाओ
धुलवानीं
चुराता
headin
केमेरो
बछड़ा
बछड़े
candy
रिलीज़
नोरिनेई
विस्फोटकों
नेपाम
ROY
खालूँ
गट्ठरों
वॉक
वॉच
वॉट
सेंकना
नापना
winging
आऊगी
बहां
बॉलरूम
Vaccum
किजिए
hm
आऊगा
KIPP
हराम

ग्लोबल
एकॉर्डियन
quench
mine
seek
शीतनीद्रा
calibrated
वेह्रमैच
उबरेंगे
झूलते
Jacques
Osjeèaš
dog
ओलंपिक
पतलू
फ़्रांस
इंजेक्षन
Free
शेरखान
मंगाई
दिलवाएंगे
डाइऑक्साइड
03
06
07
04
05
09
हैंगर
resterilized
Central
380
381
मंगाते
against
nemothorax
बिगड़ने
Carlisle
ब्लीच
सहेजा
Kalashnikovs
फायरमैन
Into
खन
आरोंपों
बहतर
कीस्मत
एम्मा
चाक़ू
make
तनहाई
जॉर्डन
vibrates
स्लाव
kid
होंऊंगा
Kobayashi
dearly
ग्रेटेल
Quilov
compared
प्रोग्राक
बैंडेज
assassination
लोंग
शील्ड
छोड़ेगा
आज़मायी
Reyniνír
च्च
फ़फ़ूंदी
dreams
shoulder
घिसकाया
तश्मे
Raiju
बँटाना
हाजिमे
eνen
फ्रेड
Sideface
उपनियमों
Shithole
2बीए
Hahahahahaaa
Knyazev
थूकना
Gwenny
बुढे
ज़ा
Handguns
छिपाई
अथॉरिटी
‚
क्रूरतापूर्ण
Santinos
Suzdržavaj
Honky
Golden
droids
एलियंस
ज़ू
अर्थतंत्रों
Knifehead
विटोरियो
रोज़ा
एपिसोड
बगान
shatterd
रोज़ी
फिस्सड्डी
नेल्सन
लॉरेन
Wii
टंगस्टन
जज़्बा
एप्रन
सबूतों
reconvene
मैग्नेट
tacos
forked
स्नाइपर
excellence
सांडोवल
लीडरों
तुड़वा
बदशगुन
खेलुंगा
लॉटन
कैप्शन
Aimed
वसान
Hellfire
Božiæe
फ़हरा
एब
एप
Continue
ringing
grouc

बेबीसिटर
जुडिथ
Rects
घबड़ाया
rappelling
cells
इनकोडिंग
कूदी
फ़ख्र
क्रिप्या
रैंत्ज़न
Like
डेकोरेटर
Crackers
फ़ांसीसियों
Pyat
हेरिंग
Sadang
गंधा
हिप्पोकैम्पस
त्रिज्या
परिखा
इज़रायल
गौफ
घिनौने
घिनौनी
1327
आंड्रूू
बतानी
लंगड़ाती
सुदृढ़
इग्लू
वाचा
Version
पड़ना
पीईटी
even
ऐबीगेल
लगोगी
लगोगे
श्राप
भविष्यद्वक्ता
ब्रोकरेज
poèistite
नैनोतकनीक
ताईवान
कौपी
रूखे
Wenneck
मरीन
आक्रामकता
स्टॉपवॉच
तैतीस
टायर्स
दांती
Seagulls
ाय
चिल
बुड्ढी
army
barks
call
calm
Screaming
चिनो
सु्प्रभात
गांएगे
Slaved
चुन्नी
भुलेखा
quot
पिऊंगा
ऑफ़िसों
answer
केसिनो
गीरा
Loverboy
Casalesi
अख़बार
purchasd
All
Zandar
Alt
आख़िरकार
जीतेंगे
baiting
कोलस्लॉ
चपके
गवा
Stomps
ऑस्बोर्न
weenis
अचर्न
गुएरा
went
afforded
तांकझांक
आटिचोक
sparkly
हिमस्खलन
Mogambo
little
स्लीपिंग
Lemoyne
बुलाएँगे
GLASS
signature
डबल्स
खटारा
anyone
clattering
ग्रीष्मकाल
जॉर्जिया
Offa
DARPA
रिंकर
पढ़ो
पढ़ी
लॉगिन
टाइटैनिक
इस्तमाल
reappoint
बियाना
दोहरानी
सिंथेटिक
पां
above
CONSTANZE
पाय
पाम
ज़ोरदार
Tallhart
एथिलीन
ड्रैक्स
शांतिप्रिय
said
reassigned
ओटावा
क्लेग


In [32]:
hi_vectors[0] *= 0
en_vectors[0] *= 0

In [33]:
en_vectors = np.array(en_vectors)
hi_vectors = np.array(hi_vectors)

In [34]:
en_vectors = en_vectors.reshape(en_vectors.shape[0],300)
hi_vectors = hi_vectors.reshape(hi_vectors.shape[0],300)

In [35]:
#en_w2v.wv.get_vector('the')

In [36]:
#en_vectors[en_word2idx['the']]

------------------

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

In [38]:
class Hindi2EnglishDataset():
    '''
        French and associated English sentences.
    '''
    
    def __init__(self, hi_sentences, en_sentences, hi_word2idx, en_word2idx, seq_length):
        self.hi_sentences = hi_sentences
        self.en_sentences = en_sentences
        self.hi_word2idx = hi_word2idx
        self.en_word2idx = en_word2idx
        self.seq_length = seq_length
        self.unk_en = set()
        self.unk_hi = set()
    
    def __len__(self):
        return len(hindi_sentences)
    
    def __getitem__(self, idx):
        '''
            Returns a pair of tensors containing word indices
            for the specified sentence pair in the dataset.
        '''
        
        # init torch tensors, note that 0 is the padding index
        hindi_tensor = torch.zeros(self.seq_length, dtype=torch.long)
        english_tensor = torch.zeros(self.seq_length, dtype=torch.long)
        
        # Get sentence pair
        hindi_sentence = self.hi_sentences[idx].split()
        english_sentence = self.en_sentences[idx].split()
        
        # Add <start> and <end> tags
        hindi_sentence.insert(0,'<start>')
        #english_sentence.insert(0,'<start>')
        hindi_sentence.append('<end>')
        english_sentence.append('<end>')
        
        # Load word indices
        for i, word in enumerate(hindi_sentence):
            if (word in hi_word2idx and hi_word_count[word] > 5):
                hindi_tensor[i] = hi_word2idx[word]
            else:
                hindi_tensor[i] = hi_word2idx['<unk>']
                self.unk_hi.add(word)
        
        for i, word in enumerate(english_sentence):
            if word in en_word2idx and en_word_count[word] > 5:
                english_tensor[i] = en_word2idx[word]
            else:
                english_tensor[i] = en_word2idx['<unk>']
                self.unk_en.add(word)
            
        sample = {'hindi_tensor': hindi_tensor, 'hindi_sentence': self.hi_sentences[idx],
                  'english_tensor': english_tensor, 'english_sentence': self.en_sentences[idx]}
        return sample

In [39]:
hindi_english_dataset = Hindi2EnglishDataset(hindi_sentences,
                                               english_sentences,
                                               hi_word2idx,
                                               en_word2idx,
                                               seq_length = seq_length)

In [40]:
test_sample = hindi_english_dataset[-10] # get 10th to last item in dataset

In [41]:
print('Input example:')
print('Sentence:')
print(test_sample['hindi_sentence'])
print('Tensor:')
print test_sample['hindi_tensor']

print('\nTarget example:')
print('Sentence:')
print test_sample['english_sentence']
print('Tensor:')
test_sample['english_tensor']

Input example:
Sentence:
बहुत अच्छी चाल चली
Tensor:
tensor([     1,   8827,    878,   6461,  19508,      2,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0])

Target example:
Sentence:
best move you have ever made
Tensor:


tensor([   746,   4075,   7512,  14344,  10131,  19918,      2,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0])

In [42]:
# Build dataloader to check how the batching works
dataloader = DataLoader(hindi_english_dataset, batch_size=32,
                        shuffle=True, num_workers=4)

In [43]:
# Prints out 3 batches from the dataloader
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['hindi_tensor'].shape,
          sample_batched['english_tensor'].shape)
    if i_batch == 3:
        break

(0, torch.Size([32, 59]), torch.Size([32, 59]))
(1, torch.Size([32, 59]), torch.Size([32, 59]))
(2, torch.Size([32, 59]), torch.Size([32, 59]))
(3, torch.Size([32, 59]), torch.Size([32, 59]))


Exception NameError: "global name 'FileNotFoundError' is not defined" in <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f47eb8c1550>> ignored


In [44]:
for i in dataloader:
    batch = i
    break

for i in range(4):
    print('Hindi Sentence:')
    print batch['hindi_sentence'][i]
    print('English Sentence:')
    print batch['english_sentence'][i]
    print '\n'

Hindi Sentence:
तुम्हें अपने जन्मदिन पर बहुत सारे तोहफ़े मिलेंगे
English Sentence:
you will get a lot of presents on your birthday


Hindi Sentence:
मुझे पता है
English Sentence:
i know


Hindi Sentence:
जब हम बंद करते हैं हम परिसर से शराब हटा देते हैं
English Sentence:
we remove the booze from the premises when we shut down


Hindi Sentence:
क्योंकि वे भूल जाते हैं लोग एक साथ नहीं रहते
English Sentence:
people don  t stay together because they forget




-----------------

## Model

In [45]:
class EncoderBiLSTM(nn.Module):
    
    def __init__(self, hidden_size, pretrained_embeddings):
        super(EncoderBiLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.embedding_dim = pretrained_embeddings.shape[1]
        self.vocab_size = pretrained_embeddings.shape[0]
        
        self.embedding = nn.Embedding(self.vocab_size,self.embedding_dim,padding_idx=0)
        self.embedding.weight = nn.Parameter(torch.FloatTensor(pretrained_embeddings), requires_grad=True)
        self.lstm = nn.LSTM(self.embedding_dim,self.hidden_size,batch_first = True,bidirectional=True)
        
    def forward(self,x):
        out = self.embedding(x)
        out = self.lstm(out)
        return out

## test encoder

In [46]:
test_encoder = EncoderBiLSTM(5, hi_vectors)

In [47]:
# Create an input tensor of random indices
test_inputs = torch.randint(0, 50, (2, 3), dtype=torch.long)

In [48]:
test_encoder_output, test_encoder_hidden = test_encoder(test_inputs)

In [49]:
test_encoder_output.size()

torch.Size([2, 3, 10])

In [50]:
test_encoder_hidden[0].size()

torch.Size([2, 2, 5])

In [51]:
test_encoder_hidden[0]

tensor([[[ 0.2520,  0.1717,  0.2962, -0.0616, -0.2956],
         [ 0.0002,  0.5029,  0.0154, -0.0082,  0.0918]],

        [[ 0.0082, -0.0110,  0.1236, -0.2562,  0.0359],
         [ 0.0022,  0.0092, -0.0064, -0.3581, -0.0171]]])

In [52]:
test_encoder_hidden[1][0::2].squeeze(0).size()

torch.Size([2, 5])

In [53]:
test_encoder_output

tensor([[[ 0.0556, -0.1769,  0.1573, -0.0123, -0.0280,  0.0082, -0.0110,
           0.1236, -0.2562,  0.0359],
         [ 0.1343, -0.3429,  0.1461, -0.0882, -0.0114, -0.0262,  0.0140,
           0.0815, -0.2105,  0.0962],
         [ 0.2520,  0.1717,  0.2962, -0.0616, -0.2956,  0.1180,  0.0733,
           0.2797, -0.0894,  0.1726]],

        [[ 0.0002,  0.4752, -0.0036, -0.0055,  0.0004,  0.0022,  0.0092,
          -0.0064, -0.3581, -0.0171],
         [-0.0000,  0.3592,  0.1354, -0.1155,  0.1548,  0.1690,  0.0821,
          -0.0408, -0.3514, -0.0117],
         [ 0.0002,  0.5029,  0.0154, -0.0082,  0.0918,  0.0016,  0.0051,
          -0.0029, -0.2890, -0.0026]]])

---------

In [54]:
class AttnDecoderLSTM(nn.Module):
    def __init__(self, decoder_hidden_size, pretrained_embeddings, seq_length):
        super(AttnDecoderLSTM, self).__init__()
        
        self.decoder_hidden_size = decoder_hidden_size
        self.seq_length = seq_length
        self.encoder_hidden_dim = 2*decoder_hidden_size
        self.embedding_dim = pretrained_embeddings.shape[1]
        self.output_vocab_size = pretrained_embeddings.shape[0]
        
        self.embedding = nn.Embedding(self.output_vocab_size,self.embedding_dim,padding_idx=0)
        self.embedding.weight = nn.Parameter(torch.FloatTensor(pretrained_embeddings), requires_grad=True)
        
        # Construct layer that calculates attentional weights
        self.attn = nn.Linear((self.decoder_hidden_size + self.embedding_dim),self.seq_length)
        
        # Construct layer that compresses the combined matrix of the input embeddings
        # and the encoder inputs after attention has been applied
        self.attn_with_input = nn.Linear((self.embedding_dim + self.encoder_hidden_dim), self.embedding_dim)
        
        # LSTM for Decoder
        self.lstm = nn.LSTM(self.embedding_dim,self.decoder_hidden_size,batch_first = True)
        # Output layer
        self.out = nn.Linear(self.decoder_hidden_size, self.output_vocab_size)
        
    def forward(self,x,hidden,encoder_output):
        
        # Input word indices, should have dim(batch_size,1), output will be (batch_size,1,embedding_dim)
        embedded = self.embedding(x)
        
        # Calculate Attention weights
        attn_weights = F.softmax(self.attn(torch.cat((hidden[0].squeeze(0), embedded.squeeze(1)),1)),dim=1)
        attn_weights = attn_weights.unsqueeze(1) # Add dimension for batch matrix multiplication
        
        # Apply Attention weights
        attn_applied = torch.bmm(attn_weights, encoder_output)
        attn_applied = attn_applied.squeeze(1) # Remove extra dimension, dim are now (batch_size, encoder_hidden_size)
        
        # Prepare LSTM input tensor
        attn_combined = torch.cat((embedded.squeeze(1), attn_applied), 1) # Combine embedding input and attn_applied,
        lstm_input = F.relu(self.attn_with_input(attn_combined)) # pass through fully connected with ReLU
        lstm_input = lstm_input.unsqueeze(1) # Add seq dimension so tensor has expected dimensions for lstm
        output, hidden = self.lstm(lstm_input, hidden) # Output dim = (batch_size,1,decoder_hidden_size)
        output = F.log_softmax(self.out(output[:,0,:]), dim=1) # softmax over all words in vocab
        
        
        return output, hidden, attn_weights

In [55]:
# Test the decoder on sample inputs to check that the dimensions of everything is correct
test_decoder_hidden_size = 5

test_decoder = AttnDecoderLSTM(test_decoder_hidden_size, en_vectors,3)

In [56]:
input_idx = torch.tensor([hi_word2idx['<start>']]*2, dtype=torch.long)

In [57]:
input_idx = input_idx.unsqueeze_(0)
test_decoder_hidden = (test_encoder_hidden[0][1::2].contiguous(), test_encoder_hidden[1][1::2].contiguous())


In [58]:
#output, hidden, attention = test_decoder.forward(input_idx, test_decoder_hidden, test_encoder_output)
#print(output.shape)

In [60]:
#output.topk(1)[1].size()

In [61]:
encoder_lstm = EncoderBiLSTM(256, hi_vectors)
decoder_lstm = AttnDecoderLSTM(256, en_vectors, seq_length)

In [62]:
criterion = nn.NLLLoss(ignore_index=0)
encoder_optimizer = torch.optim.Adam(encoder_lstm.parameters(), lr=0.0001)
decoder_optimizer = optim.Adam(decoder_lstm.parameters(), lr=0.0001)

In [63]:
num_epoch = 10
train_losses = []
valid_losses = []

running_loss = 0.0

for epoch in range(num_epoch):
                   
    print('Epoch {}/{}'.format(epoch, num_epoch- 1))
    print('-' * 10)
    
    for i, batch in tqdm.tqdm_notebook(enumerate(dataloader)):
        input_tensor = batch['hindi_tensor']
        target_tensor = batch['english_tensor']
        #print input_tensor.size()
        input_tensor = Variable(torch.LongTensor(input_tensor))
        target_tensor = Variable(torch.LongTensor(target_tensor.transpose(0,1)))
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        encoder_output, encoder_hidden = encoder_lstm(input_tensor)
        encoder_hidden = (encoder_hidden[0][0::2],encoder_hidden[1][0::2])
        # Initialize decoder input(Start of Sentence tag) and hidden state from encoder
        decoder_input =  torch.tensor([en_word2idx['<start>']]*input_tensor.shape[0], dtype=torch.long).unsqueeze(1)
        # Step through target output sequence
        loss= 0.0
        for di in range(seq_length):
            output, decoder_hidden, attn_weights = decoder_lstm(decoder_input,encoder_hidden,encoder_output)
            # Feed output as input to next item in the sequence
            decoder_input = output.topk(1)[1]
            loss+= criterion(output, target_tensor[di])
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        running_loss += loss.data * dataloader.batch_size
    epoch_loss = running_loss / hindi_english_dataset.__len__()
    print('Epoch [{}/{}]{} Loss: {:.4f}'.format(epoch+1, num_epoch,'train', epoch_loss))

Epoch 0/9
----------


A Jupyter Widget

Process Process-9:
Process Process-10:
Process Process-11:
Process Process-12:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/shaurya/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/shaurya/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/shaurya/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/shaurya/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
    self.run()
    self.run()
  File "/home/shaurya/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/shaurya/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
  File "/home/shaurya/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/shaurya/anaconda2/lib/python2.7/mul

KeyboardInterrupt: 

  File "/home/shaurya/anaconda2/lib/python2.7/site-packages/torch/multiprocessing/queue.py", line 21, in recv
    return recv()
KeyboardInterrupt
    buf = self.recv_bytes()
    buf = self.recv_bytes()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/shaurya/anaconda2/lib/python2.7/site-packages/torch/multiprocessing/queue.py", line 21, in recv
    buf = self.recv_bytes()
KeyboardInterrupt


In [ ]:
# Save the model weights to continue later
torch.save(encoder_lstm.state_dict(), './encoder_lstm.pth')
torch.save(decoder_lstm.state_dict(), './decoder_lstm.pth')

In [ ]:
def evaluate(input_tensor, encoder, decoder):
    with torch.no_grad():
        #encoder_hidden = encoder.initHidden(1)
        encoder.eval()
        decoder.eval()

        encoder_output, encoder_hidden = encoder(input_tensor.to(device))

        decoder_input =  torch.tensor([en_word2idx['<start>']]*input_tensor.shape[0], dtype=torch.long, device=device).unsqueeze(0)
        encoder_hidden = (encoder_hidden[0][0::2],encoder_hidden[1][0::2])

        output_list = []
        attn_weight_list = np.zeros((seq_length, seq_length))
        for di in range(seq_length):
            output, decoder_hidden, attn_weights = decoder(decoder_input,
                                                           encoder_hidden,
                                                           encoder_output)

            decoder_input = output.topk(1)[1].detach()
            output_list.append(output.topk(1)[1])
            word = word2idx_en[output.topk(1)[1].item()]

            attn_weight_list[di] += attn_weights[0,0,:].cpu().numpy()
        return output_list, attn_weight_list


In [ ]:
test_hindi_english_dataset = Hindi2EnglishDataset(test_hindi,
                                               test_english,
                                               hi_word2idx,
                                               en_word2idx,
                                               seq_length = seq_length)

In [ ]:
# Build dataloader to check how the batching works
test_dataloader = DataLoader(test_hindi_english_dataset, batch_size=64, num_workers=4)

In [ ]:
input_tensor.size()

In [ ]:
def get_batch(dataloader):
    for batch in dataloader:
        return batch

In [ ]:
print('Input Sentence:')
output = ''
for index in input_tensor[0]:
    word = word2idx_hi[index.item()]
    if word != '<end>':
        output += ' ' + word
    else:
        output += ' ' + word
        print(output)
        break

print('\nTarget Sentence:')
print(' ' + batch['english_sentence'][22] + '<end>')
input_len = len(batch['hindi_sentence'][22].split())

print('\nLSTM model output:')
output = ''
for index in output_list:
    word = word2idx_en[index.item()]
    if word != '<end>':
        output += ' ' + word
    else:
        output += ' ' + word
        print(output)
        break